## Remote ID Sppoofing Attack

In [ ]:
import pickle
from typing import Literal

from config import DATA_PATH, Color
from helpers import clean
from helpers.coordinates import ENU, ENUPose, GRAPose
from planner.plans.auto import AutoPlan
from simulator import Gazebo, Simulator
from simulator.gazebo.preview import GazMarker
from simulator.vehicle import SimVehicle

clean()

## Simulation Positions

In [2]:
gra_origin = GRAPose(lat=-35.3633280, lon=149.1652241,alt=0,heading=0) 
enu_origin = ENUPose(x=0, y=0, z=gra_origin.alt, heading=gra_origin.heading) 

base_homes= ENUPose.list([(-10, 0, 0, 0),(0, -5, 0, 0)])
base_paths = [ENUPose.list([(0, 0, 5, 0),(20, 0, 5, 0)]),
              [ENUPose(0, 0, 5,0)]]

In [3]:
enu_homes = enu_origin.to_abs_all(base_homes)
gra_homes = gra_origin.to_abs_all(base_homes)
enu_wptrajs = [enu_home.to_abs_all(base_path)
    for enu_home, base_path in zip(enu_homes, base_paths)]
gra_wptrajs = [gra_home.to_abs_all(base_path)
    for gra_home, base_path in zip(gra_homes, base_paths)]

## Create Vehicles

In [ ]:
sysids = [1,255] # attacker sysid is (temporally) being hardcoded to 255
              
colors=[Color.GREEN,Color.RED]

speeds=[1.0,1.0]  # m/s
lands=[True,False]
gcs_name =  f'Multicolor_{"".join([color.emoji for color in colors])}'

vehs:list[SimVehicle] = []
for sysid, enu_home, gra_wptraj, enu_wptraj, color ,speed,land in zip(
        sysids, enu_homes, gra_wptrajs, enu_wptrajs, colors,speeds,lands):
    mission_path = DATA_PATH / f"mission_{sysid}.waypoints"
    plan = AutoPlan(
        name="simple_auto_plan",
        mission_path=str(mission_path),
    )
    plan.save_basic_mission(
        sysid=sysid,
        gra_wps=GRAPose.unpose_all(gra_wptraj),
        land=land,
        speed=speed,
    )

    veh = SimVehicle(
        sysid=sysid,
        gcs_name=gcs_name,
        plan=plan,
        color=color,
        home=enu_home,
        waypoints=ENUPose.unpose_all(enu_wptraj),
    )
    vehs.append(veh)

## Scenario configurarion

In [ ]:
fake_pos = ENU(x=0,y=0,z=5)
true_pos = enu_wptrajs[1][0].unpose()
avoidance_method: Literal['stop','naive'] = 'naive'

safety_radius = 5.0  # meters
radar_radius = 10.0  # meters

# This is temporary to visualize the scenario
with open(DATA_PATH/"fake_position.pkl", "wb") as f:
    pickle.dump(fake_pos, f)

with open(DATA_PATH/"avoid_method.txt", "w") as f:
    f.write(avoidance_method)

## Gazebo

In [6]:
gaz= Gazebo(gra_origin,world_path="simulator/gazebo/worlds/runway.world")

origin_gaz = GazMarker(name="origin",
                    group="origin",
                    pos=enu_origin.unpose(),
                    color=Color.WHITE)


true_marker = GazMarker(name="true_pos",
                    group="true_pos",
                    pos=true_pos,
                    color=Color.RED)

fake_marker = GazMarker(name="fake_pos",
                group="fake_pos",
                pos=fake_pos,
                color=Color.ORANGE)

avoid_zone = GazMarker(name="avoid_zone",
                group="avoid_zone",
                pos=fake_pos,
                color=Color.RED,
                radius=safety_radius,
                alpha=0.85)

radar_zone = GazMarker(name="radar_zone",
                group="radar_zone",
                pos=true_pos,
                color=Color.ORANGE,
                radius=radar_radius,
                alpha=0.9)

for marker in [origin_gaz, true_marker, fake_marker, avoid_zone, radar_zone]:
    gaz.markers.append(marker)

## Simulator

In [ ]:
simulator = Simulator(
	visualizer=gaz,
	terminals=['gcs'],
	verbose=1,
)
for veh in vehs:
    simulator.add_vehicle(veh)

simulator.show()

## Run

In [ ]:
orac = simulator.launch()
orac.run()

07:18:49 - Oracle ⚪ - INFO - 🖥️  Gazebo launched for realistic simulation and 3D visualization.
07:18:49 - Oracle ⚪ - INFO - 🚀 GCS Multicolor_🟩🟥 launched (PID 3894632)
07:18:49 - Oracle ⚪ - INFO - 🏁 Starting Oracle with 2 vehicles and 1 GCSs
